##Red Neuronal (2 capas) desde cero

##Importar librerias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


##Importar Mnist

In [ ]:
!pip install mnist

In [ ]:
import mnist

In [ ]:
!pip install kora -q
from kora import drive
drive.link_nbs()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('Get_Images.py','wb').write(src)
import Get_Images

In [ ]:
# Importacion de funcion get_images desde archivo Get_Images.py
from Get_Images import get_images

In [ ]:
mnist.path = '/content/drive/MyDrive/Deep Learning/mnist_raw'

In [ ]:
print(mnist.path)

In [ ]:
# Importa las variables desde el archivo Get_images.py
from Get_Images import x_train, y_train, x_test, y_test

# Crea las nuevas variables
x_train_num = x_train
y_train_num = y_train
x_test_num = x_test
y_test_num = y_test

In [ ]:
# Creacion de variables y resize de imagenes
x_train = x_train_num[:50000].reshape(50000, -1).astype(float)/255
y_train = y_train_num[:50000].reshape(50000, 1)

x_val = x_train_num[50000:].reshape(10000, -1).astype(float)/255
y_val = y_train_num[50000:].reshape(10000, 1)

x_test = x_test_num.copy().reshape(10000, -1).astype(float)/255
y_test = y_test_num.copy().reshape(10000, 1)

In [ ]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

print(x_test.shape)
print(y_test.shape)

##Graficar algunas imagenes

In [ ]:
def plot_number(image):
  plt.imshow(image.squeeze(), cmap=plt.get_cmap('gray'))
  plt.axis('off')
  plt.show()

In [ ]:
                            ##Busqueda de nro random
rnd_idx = np.random.randint(len(y_test))
                            ##Comentario
print(f'La imagen muestreada representa un: {y_test[rnd_idx]}')
                            ##Importacion de imagen
plot_number(x_test_num[rnd_idx])

##Red Neuronal

##Dos capas 200-10

##Crear mini batches

In [ ]:
x_train.shape[0]

In [ ]:
                             ##Creacion de minibatches##

def create_minibatches(mb_size, x, y, shuffle = True):

    # x  #muestras, 784
    # y #muestras, 1

    assert x.shape[0] == y.shape[0], 'Error en cantidad de muestras'
    total_data = x.shape[0]
    if shuffle:
        idxs = np.arange(total_data)
        np.random.shuffle(idxs)
        x = x[idxs]
        y = y[idxs]

    return ((x[i:i+mb_size], y[i:i+mb_size]) for i in range(0, total_data, mb_size))

##Init parameters

In [ ]:
def init_parameters(input_size, neurons):

    # input_size -> elementos de entrada, 784
    # neurons -> list [200, 10] con cantidad de neuronas en cada capa

    W1 = np.random.randn(neurons[0], input_size) * 0.001
    b1 = np.zeros((neurons[0], 1))

    W2 = np.random.randn(neurons[1], neurons[0]) * 0.001
    b2 = np.zeros((neurons[1], 1))

    return {'W1': W1, 'b1':b1, 'W2':W2, 'b2':b2}


In [ ]:
#Prueba de parametros
parameters = init_parameters(28*28, [200, 10])
print(parameters['W1'].shape)
print(parameters['W2'].shape)
print(parameters['b2'].shape)

In [ ]:
def relu(x):
  return np.maximum(0, x)

In [ ]:
def linear_pass(x, parameters, activation_fcn):

    #¤ x tiene la forma (#pixeles, num samples)

    z1 = parameters['W1'] @ x + parameters['b1']
    a1 = activation_fcn(z1) # devuel fcn. de activa.
    z2 = parameters['W2'] @ a1 + parameters['b2']

    return z2, z1, a1

In [ ]:
scores, z1, a1 = linear_pass(x_train[:64].T, parameters, relu)

In [ ]:
x_train[:64].shape

In [ ]:
scores.shape

In [ ]:
def softmax(x):
    exp_scores = np.exp(x)
    sum_exp_scores = np.sum(exp_scores, axis=0)
    probs = exp_scores/sum_exp_scores
    return probs

In [ ]:
def x_entropy(scores, y, batch_size=64):
    probs = softmax(scores)
    y_hat = probs[y.squeeze(), np.arange(batch_size)]
    cost = np.sum(-np.log(y_hat)) / batch_size

    return probs, cost

In [ ]:
def backward(probs, x, y, z1, a1, scores, parameters, batch_size=64):
    grads = {}
    probs[y.squeeze(), np.arange(batch_size)] -= 1 # y-hat - y
    dz2 = probs.copy()

    dW2 = dz2 @ a1.T / batch_size
    db2 = np.sum(dz2, axis =1, keepdims=True) / batch_size
    da1 = parameters['W2'].T @ dz2

    dz1 = da1.copy()
    dz1[z1 <= 0 ] =0

    dW1 = dz1 @ x
    db1 = np.sum(dz1, axis=1, keepdims=True)

    assert parameters['W1'].shape == dW1.shape, 'W1 no igual forma'
    assert parameters['W2'].shape == dW2.shape, 'W2 no igual forma'
    assert parameters['b1'].shape == db1.shape, 'b1 no igual forma'
    assert parameters['b2'].shape == db2.shape, 'b2 no igual forma'

    grads = {'w1':dW1,  'b1':db1, 'W2':dW2, 'b2':db2}

    return grads

In [ ]:
y_hat, cost = x_entropy(scores, y_train[:64])

In [ ]:
y_hat.shape

In [ ]:
print(cost)

In [ ]:
grads = backward(y_hat, x_train[:64], y_train[:64],z1, a1, scores, parameters)

In [ ]:
def accuracy(x_data, y_data, mb_size=64):
    correct = 0
    total = 0
    for i, (x, y) in enumerate(create_minibatches(mb_size, x_data, y_data)):
        scores2, z1, a1 = linear_pass(x.T, parameters, relu)
        y_hat, cost = x_entropy(scores2, y, batch_size=len(x))

        correct += np.sum(np.argmax(y_hat, axis=0) == y.squeeze())
        total += y_hat.shape[1]
    return correct/total

In [ ]:
def train(epochs, parameters, mb_size=64, learning_rate = 1e-3):
    for epoch in range(epochs):
        for i, (x, y) in enumerate(create_minibatches(mb_size, x_train, y_train)):
            scores2, z1, a1 = linear_pass(x.T, parameters=parameters, activation_fcn=relu)
            y_hat, cost = x_entropy(scores2, y, batch_size=len(x))
            grads = backward(y_hat, x, y, z1, a1, scores2, parameters, batch_size=len(x))

            parameters['W1'] = parameters['W1'] - learning_rate*grads['w1']
            parameters['b1'] = parameters['b1'] - learning_rate*grads['b1']
            parameters['b2'] = parameters['b2'] - learning_rate*grads['b2']
            parameters['W2'] = parameters['W2'] - learning_rate*grads['W2']

        print(f'costo es: {cost}, y accuracy: {accuracy(x_val, y_val, mb_size)}')
    return parameters

In [ ]:
print(type(scores))

In [ ]:
mb_size = 512
learning_rate = 1e-2
epochs = 20
parameters = train(epochs=epochs, parameters=parameters, mb_size=mb_size, learning_rate=learning_rate)

In [ ]:
accuracy(x_train, y_train, mb_size)

In [ ]:
accuracy(x_test, y_test, mb_size)

In [ ]:
def predict(x):
    scores2, _, _ = linear_pass(x, parameters, relu)
    return np.argmax(scores2)

In [ ]:
x_test[0].reshape(-1, 1).shape

In [ ]:
idx = np.random.randint(len(y_test))
plot_number(x_test_num[idx])
pred = predict(x_test[idx].reshape(-1, 1))
print(f'el valor predicho es: {pred}')